In [1]:
pip install CTGANENN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sdmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu1

In [2]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

dataset = pd.read_csv('https://raw.githubusercontent.com/mahayasa/gan-hybrid-sampling-customer-churn/main/data/telco_new.csv')

df1=dataset[dataset['Churn']==0]
df=dataset[dataset['Churn']==1]

In [3]:
from ctganenn import CTGANENN

X,y=CTGANENN(df,df1,2655,'Churn')

Gen. (-0.96) | Discrim. (-0.11): 100%|██████████| 100/100 [01:00<00:00,  1.66it/s]


In [4]:
# define model
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
#balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.853563 using {'class_weight': {0: 100, 1: 100}}
0.813980 (0.015434) with: {'class_weight': {0: 10, 1: 100}}
0.827228 (0.015805) with: {'class_weight': {0: 20, 1: 100}}
0.837707 (0.022499) with: {'class_weight': {0: 30, 1: 100}}
0.838709 (0.015968) with: {'class_weight': {0: 40, 1: 100}}
0.842412 (0.016386) with: {'class_weight': {0: 50, 1: 100}}
0.845298 (0.016304) with: {'class_weight': {0: 60, 1: 100}}
0.847886 (0.016250) with: {'class_weight': {0: 70, 1: 100}}
0.849215 (0.016433) with: {'class_weight': {0: 80, 1: 100}}
0.851033 (0.016533) with: {'class_weight': {0: 90, 1: 100}}
0.853563 (0.016316) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('LR-telco3-ctgan-auc-fn.xlsx', index=False)

In [6]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.768268 using {'class_weight': {0: 70, 1: 100}}
0.667603 (0.014550) with: {'class_weight': {0: 10, 1: 100}}
0.704788 (0.015399) with: {'class_weight': {0: 20, 1: 100}}
0.735251 (0.018371) with: {'class_weight': {0: 30, 1: 100}}
0.748181 (0.016979) with: {'class_weight': {0: 40, 1: 100}}
0.757792 (0.017830) with: {'class_weight': {0: 50, 1: 100}}
0.765193 (0.019057) with: {'class_weight': {0: 60, 1: 100}}
0.768268 (0.016381) with: {'class_weight': {0: 70, 1: 100}}
0.766599 (0.017705) with: {'class_weight': {0: 80, 1: 100}}
0.766920 (0.015279) with: {'class_weight': {0: 90, 1: 100}}
0.766252 (0.017486) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('LR-telco3-ctgan-f1-fn.xlsx', index=False)

In [8]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.772090 using {'class_weight': {0: 70, 1: 100}}
0.653612 (0.016823) with: {'class_weight': {0: 10, 1: 100}}
0.698631 (0.017201) with: {'class_weight': {0: 20, 1: 100}}
0.734332 (0.019577) with: {'class_weight': {0: 30, 1: 100}}
0.750284 (0.017405) with: {'class_weight': {0: 40, 1: 100}}
0.761479 (0.018091) with: {'class_weight': {0: 50, 1: 100}}
0.769347 (0.019154) with: {'class_weight': {0: 60, 1: 100}}
0.772090 (0.016359) with: {'class_weight': {0: 70, 1: 100}}
0.769410 (0.017768) with: {'class_weight': {0: 80, 1: 100}}
0.768295 (0.015270) with: {'class_weight': {0: 90, 1: 100}}
0.765909 (0.017348) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('LR-telco3-ctgan-gmean-fn.xlsx', index=False)